In [1]:
import pickle
from glob import glob
import numpy as np
import os
from tqdm import tqdm

In [2]:
pkl_list = glob(os.path.join('train', '*'))
len(pkl_list)

205942

In [61]:
X = []
y = []
lane = []
others = []

for pkl_path in tqdm(pkl_list):
    with open(pkl_path, 'rb') as f:
        scene = pickle.load(f)
    
    # the index of agent to be predicted 
    pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
    
    p_in_raw = scene['p_in'][pred_id]
    p_out_raw = scene['p_out'][pred_id]
    v_in_raw = scene['v_in'][pred_id]
    v_out_raw = scene['v_out'][pred_id]
#     print(v_in_raw)
    
    lane_scene = scene['lane']
    
    # Normalization
    min_vecs = np.min(lane_scene, axis = 0)
    max_vecs = np.max(lane_scene, axis = 0)
    
    # Normalize by vectors
    p_in_normalized = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
    p_out_normalized = (p_out_raw - min_vecs)/(max_vecs - min_vecs)
    v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
    v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
    v_in_normalized = v_in_raw / v_in_norm
    v_out_norm = np.linalg.norm(v_out_raw, axis=1, keepdims=True)
    v_out_norm = np.where(v_out_norm == 0.0, 1.0, v_out_norm)
    v_out_normalized = v_out_raw / v_out_norm
    inp = np.concatenate((p_in_normalized,v_in_normalized),axis=1)
    output = np.concatenate((p_out_normalized, v_out_normalized), axis=1)
    
#     print(v_in_norm.shape)
    
    p_in_other_raw = np.array([scene['p_in'][i] for i in range(60) if scene['car_mask'][i] and i != pred_id])
    p_in_other_normalized = (p_in_other_raw- min_vecs)/(max_vecs - min_vecs)
    v_in_other_raw = np.array([scene['v_in'][i] for i in range(60) if scene['car_mask'][i] and i != pred_id])
    v_in_other_norm = np.linalg.norm(v_in_other_raw, axis=2, keepdims=True)
    v_in_other_norm = np.where(v_in_other_norm == 0.0, 1.0, v_in_other_norm)
    v_in_other_normalized = v_in_other_raw / v_in_other_norm
    other = np.concatenate((p_in_other_normalized,v_in_other_normalized),axis=2)
    other = other.reshape((other.shape[0], -1))
    if len(other) < 20:
        other = np.concatenate((other, np.zeros((20-len(other), 76))), axis=0)
    other = other[:20]
    
    lane_scene = (lane_scene - min_vecs)/(max_vecs - min_vecs)
    if len(lane_scene) < 1000:
        lane_scene = np.concatenate((lane_scene, np.zeros((1000-len(lane_scene), 2))), axis=0)
    lane_scene = lane_scene[:1000]

    X.append(inp)
    y.append(output)
    lane.append(lane_scene)
    others.append(other)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1123.37it/s]


In [63]:
npx = np.stack(X)
npx.shape

(1000, 19, 4)

In [64]:
npl = np.stack(lane)
npl.shape

(1000, 1000, 2)

In [65]:
npo = np.stack(others)
npo.shape

(1000, 20, 76)

In [66]:
npy = np.stack(y)
npy = npy[..., :2]
npy.shape

(1000, 30, 2)

In [68]:
with open('train_X.npy', 'wb') as f:
    np.save(f, npx)

with open('train_y.npy', 'wb') as f:
    np.save(f, npy)

with open('train_lane.npy', 'wb') as f:
    np.save(f, npl)
    
with open('train_other.npy', 'wb') as f:
    np.save(f, npo)